In [1]:
import pandas as pd

In [2]:
file_path = "./data/data.csv"
data = pd.read_csv(file_path)
data.head()

,time,V_avg_machine,I_avg_machine,kW_machine,kvar_machine,kVA_machine,PF_machine,kWh_machine,kvarh_machine,kVAh_machine,...,I_avg_spindle,kW_spindle,kvar_spindle,kVA_spindle,PF_spindle,kWh_spindle,kvarh_spindle,kVAh_spindle,RPM,Anomaly
0,2023-05-04 21:06:25,222.555862,3.978290,1.184035,0.968490,1.529677,0.774042,1.816337,1.503314,2.358233,...,5.533978,0.152449,2.475371,2.480061,0.061470,0.164464,1.580153,1.598300,2500,False
1,2023-05-04 21:06:26,222.595947,3.980162,1.184236,0.968880,1.530079,0.773970,1.816667,1.503583,2.358658,...,5.545718,0.152649,2.477063,2.481762,0.061508,0.164549,1.581529,1.599683,2500,False
2,2023-05-04 21:06:27,222.631027,3.984233,1.184538,0.968671,1.530181,0.774116,1.816996,1.503852,2.359083,...,5.550326,0.152649,2.478667,2.483363,0.061469,0.164591,1.582217,1.600376,2500,False
3,2023-05-04 21:06:28,222.631027,3.984233,1.184538,0.968671,1.530181,0.774116,1.816996,1.503852,2.359083,...,5.550326,0.152649,2.478667,2.483363,0.061469,0.164591,1.582217,1.600376,2500,False
4,2023-05-04 21:06:29,222.526733,3.982668,1.184841,0.970051,1.531290,0.773754,1.817654,1.504391,2.359936,...,5.559220,0.152848,2.482263,2.486965,0.061460,0.164677,1.583596,1.601762,2500,False
